In [2]:
import pandas as pd
import numpy as np
import math 

#load questions csv
questions_data = pd.read_csv('../OkCupidData/question_data.csv',header=0,delimiter=';' )
questions_data = questions_data.rename(columns={questions_data.columns[0]: "q" })
#keep only question lines
questions_data = questions_data[questions_data.q.str[0] == 'q']
#sort them by non empty number of lines
questions_data.sort_values(by=['N'],ascending=False)
orig_data = pd.read_pickle("../OkCupidData/unprocessed.bz2")

###split data in questions, demographics, probabilities###
prob_data = orig_data[[col for col in orig_data.columns if col.startswith('p')]]
question_data = orig_data[[col for col in orig_data.columns if col.startswith('q')]]
demog_data = orig_data[[col for col in orig_data.columns if not col.startswith('q') and not col.startswith('p')]]
demog_question_data = orig_data[[col for col in orig_data.columns if not col.startswith('p')]]

In [26]:
#these dimensions are complementary, need to swap them 
indx_p_dominant = prob_data.columns.get_loc("p_dominant")
indx_p_submissive = prob_data.columns.get_loc("p_submissive")

def user_similarity(user_1_indx, user_2_indx, prob_data=prob_data):
    values_1 = prob_data.iloc[[user_1_indx]].values[0]
    values_2 = prob_data.iloc[[user_2_indx]].values[0]
    values_2[indx_p_dominant], values_2[indx_p_submissive] =  values_2[indx_p_submissive], values_2[indx_p_dominant] 
    
    sum_sq_1 = np.nansum(values_1*values_1)
    if sum_sq_1 == 0:
        return 0
    sum_sq_2 = np.nansum(values_2*values_2)
    if sum_sq_2 == 0:
        return 0
    return np.nansum(values_1*values_2) / (np.sqrt(sum_sq_1) * np.sqrt(sum_sq_2))

# user_similarity(2711, 390)

0.09962332051936486

In [32]:
def test_similarity_metric():
    #find closest vector
    max_val = 0
    for i in range(len(prob_data)):
        cosine_sim = user_similarity(2711, i)
        if cosine_sim > max_val:
            print cosine_sim
            max_val = cosine_sim
            display(prob_data.iloc[[i]])
# test_similarity_metric()

50